In [47]:
import time
import sys
import numpy as np
import skimage
from skimage import data, transform
import matplotlib.pyplot as plt
from skimage import io, exposure, img_as_uint, img_as_float
import pickle
import time
import math
import keras
from keras import *
from keras.layers import Input, Dense, Activation
from keras.models import Model, Sequential
from sklearn.utils import shuffle



In [2]:

encoding_dim = 4  # 4 angles are learned for 4 joints
decoding_dim = 19200 # Images are downsized to 210 * 120


# I had a black first image at first even though there was an image
n_samples = 10001
# Create n number of frames, for sequence for one angle
interpolation_steps = 4

with open('angles-observed.pkl', 'rb') as pickle_file:
    angles = pickle.load(pickle_file)
print("Loaded observed:" + str(len(angles)) + " angles")

lst_images = list()
lst_angles = list()

Loaded observed:40404 angles


In [3]:
for i in range(4, (len(angles) - 400)):
    # Load angles
    lst_angles.append(angles[i])
    image = skimage.io.imread('C:\\Users\\skork\\Documents\\img\\depth_' + str(i // 4) + "_" + str(i % 4) +  ".png" , as_grey=True)
    image_rescaled = skimage.transform.rescale(image, 1.0 / 4.0)
    lst_images.append(image_rescaled)


X = np.asarray(lst_images)
Y = np.asarray(lst_angles)


c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [59]:
for i in range(5):
    np.random.seed(i)
    np.random.shuffle(X)
    np.random.seed(i)
    np.random.shuffle(Y)
   
    training, test = X[:32000,:], X[32000:40000,:]
    training_angle, test_angle = Y[:32000,:], Y[32000:40000,:]


In [18]:
angles_dim = 4  #  floats -> compression of factor 24.5, assuming the input is 784 floats

x_train = X.reshape((len(X), np.prod(X.shape[1:])))


input_img = Input(shape=(19200,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(19200, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (4-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_train, x_train))

Epoch 1/50
40000/40000 [==============================] - 32s 795us/step - loss: 0.6922
Epoch 2/50
40000/40000 [==============================] - 26s 654us/step - loss: 0.6790
Epoch 3/50
40000/40000 [==============================] - 26s 652us/step - loss: 0.33892s 
Epoch 4/50
40000/40000 [==============================] - 26s 661us/step - loss: 0.0988
Epoch 5/50
40000/40000 [==============================] - 26s 650us/step - loss: 0.0822
Epoch 6/50
40000/40000 [==============================] - 26s 658us/step - loss: 0.0783
Epoch 7/50
40000/40000 [==============================] - 26s 655us/step - loss: 0.0766
Epoch 8/50
40000/40000 [==============================] - 26s 646us/step - loss: 0.0757
Epoch 9/50
40000/40000 [==============================] - 26s 645us/step - loss: 0.0752
Epoch 10/50
40000/40000 [==============================] - 26s 651us/step - loss: 0.0748
Epoch 11/50
40000/40000 [==============================] - 26s 655us/step - loss: 0.0745
Epoch 12/50
40000/40000 [==

KeyboardInterrupt: 

In [ ]:
x_train = training.reshape((len(training), np.prod(training.shape[1:])))
y_train = training_angle.reshape((len(training_angle), np.prod(training_angle.shape[1:])))

x_test = test.reshape((len(test), np.prod(test.shape[1:])))
y_test = test_angle.reshape((len(test_angle), np.prod(test_angle.shape[1:])))

model = Sequential()
model.add(Dense(1000, input_shape=(19200,)))
model.add(Activation('relu'))
model.add(Dense(100, input_shape=(1000,)))
model.add(Activation('relu'))
model.add(Dense(4))
model.compile(loss='mean_squared_error', optimizer='Adam')
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Train on 32000 samples, validate on 8000 samples
Epoch 1/20
32000/32000 [==============================] - 63s 2ms/step - loss: 0.0456 - val_loss: 0.0422
Epoch 2/20
32000/32000 [==============================] - 61s 2ms/step - loss: 0.0414 - val_loss: 0.0420
Epoch 3/20
32000/32000 [==============================] - 61s 2ms/step - loss: 0.0413 - val_loss: 0.0421
Epoch 4/20
 2752/32000 [=>............................] - ETA: 54s - loss: 0.0420

In [14]:
x_train = X.reshape((len(X), np.prod(X.shape[1:])))


array([[0.12054431, 0.12054431, 0.12054431, ..., 0.12054431, 0.12054431,
        0.12054431],
       [0.1164149 , 0.1164149 , 0.1164149 , ..., 0.1164149 , 0.1164149 ,
        0.1164149 ],
       [0.11256078, 0.11256078, 0.11256078, ..., 0.11256078, 0.11256078,
        0.11256078],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [48]:
X, y = shuffle(X, Y, random_state=0)


In [60]:
test_angle.shape

(8000, 4)

In [62]:
model = None